In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline


In [2]:
data = pd.read_csv('/content/drive/MyDrive/merged_files_enron123.csv')

In [3]:
data.head()

,text,label
0,"Subject: what up , , your cam babe\nwhat are y...",spam
1,Subject: want to make more money ?\norder conf...,spam
2,Subject: food for thoughts\n[\njoin now - take...,spam
3,Subject: miningnews . net newsletter - tuesday...,spam
4,Subject: your pharmacy ta\nwould you want chea...,spam


In [4]:
data.rename(columns = {'text':'text', 'label':'spam'},
                         inplace = True)

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['spam'] = le.fit_transform(data['spam'])

In [6]:
data['spam'].value_counts()

0    12045
1     4496
Name: spam, dtype: int64

In [7]:
X = data['text']
y = data['spam']

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size=0.2)

In [9]:
!pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00


In [10]:
from keras_preprocessing.sequence import pad_sequences

In [11]:
max_words = 100000
max_len = 250
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences, maxlen=max_len)

In [12]:
def RNN():
  inputs = Input(name='inputs', shape=[max_len])
  layer = Embedding(max_words, 70, input_length=max_len)(inputs)
  layer = LSTM(64)(layer)
  layer = Dense(256, name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(0.5)(layer)
  layer = Dense(1, name='out_layer')(layer)
  layer = Activation('sigmoid')(layer)
  model = Model(inputs=inputs,outputs=layer)
  return model

In [13]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 250)]             0         
                                                                 
 embedding (Embedding)       (None, 250, 70)           7000000   
                                                                 
 lstm (LSTM)                 (None, 64)                34560     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [14]:
model.fit(sequences_matrix, Y_train, batch_size=10, epochs=10, 
          validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0001)])

Epoch 1/10
1059/1059 [==============================] - 291s 272ms/step - loss: 0.1313 - accuracy: 0.9469 - val_loss: 0.0552 - val_accuracy: 0.9807
Epoch 2/10
1059/1059 [==============================] - 275s 260ms/step - loss: 0.0177 - accuracy: 0.9946 - val_loss: 0.0506 - val_accuracy: 0.9856
Epoch 3/10
1059/1059 [==============================] - 268s 253ms/step - loss: 0.0086 - accuracy: 0.9979 - val_loss: 0.0482 - val_accuracy: 0.9853
Epoch 4/10
1059/1059 [==============================] - 272s 257ms/step - loss: 0.0018 - accuracy: 0.9993 - val_loss: 0.0677 - val_accuracy: 0.9807


In [15]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences, maxlen=max_len)

In [16]:
accuracy = model.evaluate(test_sequences_matrix, Y_test)

104/104 [==============================] - 4s 41ms/step - loss: 0.1040 - accuracy: 0.9761


In [17]:
y_pred = model.predict(test_sequences_matrix)

104/104 [==============================] - 6s 56ms/step


In [18]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, classification_report

In [19]:
y_pred

array([[2.6386955e-09],
       [4.7305780e-06],
       [8.4215953e-06],
       ...,
       [2.2906644e-08],
       [9.9753940e-01],
       [1.8809428e-02]], dtype=float32)

In [20]:
len(y_pred)

3309

In [21]:
for i in range(len(y_pred)):
  if y_pred[i] > 0.4:
    y_pred[i] = 1

  else:
    y_pred[i] =0


In [22]:
y_pred = y_pred.T

In [23]:
print(classification_report(Y_test, y_pred[0]))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      2411
           1       0.99      0.93      0.96       898

    accuracy                           0.98      3309
   macro avg       0.98      0.96      0.97      3309
weighted avg       0.98      0.98      0.98      3309



In [24]:
print(confusion_matrix(Y_test, y_pred[0]))


[[2399   12]
 [  62  836]]


In [25]:
print(accuracy_score(Y_test, y_pred[0]))

0.977636748262315


In [26]:
Testing_K = ["""Say, do you remember where you spent your last month's salary/income?

Maybe you feel like keeping a budget and tracking every expense is extremely difficult.

 

We agree - it is not easy, certainly not to do consistently.

 

Luckily, you no longer have to depend on spreadsheets, complex financial tools, your old ledgers, or even your memory to do this anymore. In this day and age of technology, many apps can help you do that."""]

In [27]:
txts = tok.texts_to_sequences(Testing_K)
txts = pad_sequences(txts, maxlen=max_len)

In [28]:
preds = model.predict(txts)
print(preds)

1/1 [==============================] - 0s 32ms/step
[[0.8812163]]
